
system usage report?


In [ ]:

const os = require('os')


const reportCPU = []
const reportMem = []

async function systemUsage() {
  const cpus = os.cpus()
  let cpuTotal = 0

  for(var i = 0, len = cpus.length; i < len; i++) {
    var cpu = cpus[i], total = 0;

    for(var type in cpu.times) {
        total += cpu.times[type];
    }

    cpus[i].usage = {}
    for(type in cpu.times) {
      cpus[i].usage[type] = Math.round(100 * cpu.times[type] / total);
    }
    
    cpuTotal += cpus[i].usage.sys
  }

  reportMem.unshift(Math.round(100 * (os.totalmem() - os.freemem()) / os.totalmem()))
  reportCPU.unshift(Math.round(cpuTotal / cpus.length))

  if(reportMem.length > 100 || reportCPU.length > 100) {
    reportMem.pop()
    reportCPU.pop()
  }

  if(reportCPU.length == 1) {
    return await new Promise(resolve => setTimeout(() => resolve(systemUsage()), 1000))
  }

  return {
    memory: reportMem,
    cpus: reportCPU
  }
}

module.exports = systemUsage



system usage graph?


In [ ]:

import { D3Node } from 'd3-node'
const systemUsage = importer.import('system usage report')

async function graphUsage(data) {

  if(!data) {
    let usage = await systemUsage()
    let now = Date.now()
    data = usage.cpus.map((t, i) => ({
      n: t,
      year: now - i * 1000,
      name: 'cpu'
    })).concat(usage.memory.map((t, i) => ({
      n: t,
      year: now - i * 1000,
      name: 'mem'
    }))).sort((a, b) => a.name - b.name)

    console.log(data)
  }

  // set the dimensions and margins of the graph
  const margin = {top: 10, right: 30, bottom: 30, left: 60},
      width = 460 - margin.left - margin.right,
      height = 200 - margin.top - margin.bottom;

  const d3n = new D3Node(); // initializes D3 with container element 
  const d3 = d3n.d3;

  // append the svg object to the body of the page
  const svg = d3n.createSVG(
    width + margin.left + margin.right,
    height + margin.top + margin.bottom)
  /*const svg = d3.select("#my_dataviz")
    .append("svg")
      .attr("width", width + margin.left + margin.right)
      .attr("height", height + margin.top + margin.bottom)
  */
    .append("g")
      .attr("transform", `translate(${margin.left},${margin.top})`);

  // group the data: I want to draw one line per group
  const sumstat = d3.group(data, d => d.name); // nest function allows to group the calculation per level of a factor

  // Add X axis --> it is a date format
  const x = d3.scaleLinear()
    .domain(d3.extent(data, function(d) { return d.year; }))
    .range([ 0, width ]);
  svg.append("g")
    .attr("transform", `translate(0, ${height})`)
    .call(d3.axisBottom(x).ticks(5));

  // Add Y axis
  const y = d3.scaleLinear()
    .domain([0, 100 /* d3.max(data, function(d) { return +d.n; }) */])
    .range([ height, 0 ]);
  svg.append("g")
    .call(d3.axisLeft(y));

  // color palette
  const color = d3.scaleOrdinal()
    .range(['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf','#999999'])

  // Draw the line
  svg.selectAll(".line")
      .data(sumstat)
      .join("path")
        .attr("fill", "none")
        .attr("stroke", function(d){ return color(d[0]) })
        .attr("stroke-width", 1.5)
        .attr("d", function(d){
          return d3.line()
            .x(function(d) { return x(d.year); })
            .y(function(d) { return y(+d.n); })
            (d[1])
        })

  return d3n.svgString();
}

export default graphUsage

